In [1]:
require 'hdf5'

{
  H5Z_FILTER_CONFIG_ENCODE_ENABLED : 1
  H5F_ACC_RDWR : 1
  _getTorchType : function: 0x04785a98
  H5F_OBJ_FILE : 1
  H5S_ALL : 0
  H5F_OBJ_GROUP : 4
  C : userdata: 0x04a70c50
  H5P_DEFAULT : 0
  _describeObject : function: 0x047bc430
  H5Z_FILTER_NBIT : 5
  _debugMode : false
  _getObjectType : function: 0x047bc478
  H5F_OBJ_ALL : 31
  _getObjectName : function: 0x047da038
  version : 
    {
      1 : 1
      2 : 8
      3 : 15
    }
  H5Z_FILTER_SHUFFLE : 2
  HDF5Group : table: 0x047c00f8
  open : function: 0x04a76f48
  H5Z_FILTER_SZIP : 4
  H5F_OBJ_ATTR : 16
  H5Z_FILTER_FLETCHER32 : 3
  H5F_OBJ_DATATYPE : 8
  debugMode : function: 0x0479dcf0
  H5F_ACC_EXCL : 4
  H5Z_FILTER_NONE : 0
  _testUtils : 
    {
      deepAlmostEq : function: 0x0479dcb0
      withTmpDir : function: 0x047d5578
    }
  _nativeTypeForTensorType : function: 0x04d13fd0
  H5F_ACC_TRUNC : 2
  _config : 
    {
      HDF5_INCLUDE_PATH : /Users/virgileaudi/anaconda/include
      HDF5_LIBRARIES : /Users/virgileaudi

      NATIVE_INT32 : 50331676
      VLEN : 9
      ARRAY : 10
      STD_U16LE : 50331718
      STD_B16LE : 50331726
      STD_I64LE : 50331714
      NATIVE_UINT16 : 50331671
      NATIVE_UINT_FAST8 : 50331669
      STD_B16BE : 50331727
      NATIVE_INT_FAST8 : 50331668
      FLOAT : 1
      REFERENCE : 7
      STD_U32LE : 50331720
      NATIVE_USHORT : 50331659
      NATIVE_ULLONG : 50331689
      NATIVE_INT8 : 50331664
      IEEE_F32LE : 50331702
      STD_U8BE : 50331717
      NATIVE_INT_LEAST8 : 50331666
      NATIVE_UINT8 : 50331665
      NATIVE_B32 : 50331695
      NATIVE_HERR : 50331700
      NATIVE_OPAQUE : 50331736
      NATIVE_LDOUBLE : 50331692
      NATIVE_UINT_LEAST8 : 50331667
      COMPOUND : 6
      STD_REF_OBJ : 50331739
      NATIVE_UINT_FAST16 : 50331675
      NATIVE_B64 : 50331696
      STD_U16BE : 50331719
      STD_REF_DSETREG : 50331740
      NATIVE_B8 : 50331693
      STD_I32BE : 50331713
      IEEE_F64BE : 50331705
      NATIVE_FLOAT : 50331690
      NATIVE_UCHA

In [2]:
myFile = hdf5.open('SST1.hdf5','r')
og_train_input = myFile:read('train_input'):all()
og_test_input = myFile:read('test_input'):all()
nfeatures = myFile:read('nfeatures'):all()
og_valid_input = myFile:read('valid_input'):all()
og_train_output = myFile:read('train_output'):all()
og_valid_output = myFile:read('valid_output'):all()
nclasses = myFile:read('nclasses'):all()
myFile:close()

# Cross-validation LogReg:

In [3]:
function logreg(Xtrain,Ytrain,nfeatures,ep_max,m,eta,lambda)
    
    local shuffle = torch.LongTensor(Xtrain:size(1))
    
    local W = torch.cat(torch.zeros(1,5),torch.ones(nfeatures[1],5),1)
    local b = torch.ones(5):view(-1,1)

    local grad_W = torch.zeros(nfeatures[1]+1,5)
    local grad_b = torch.zeros(5):view(-1,1)

    local z = torch.ones(5):view(-1,1)
    local yhat = torch.ones(5):view(-1,1)
    local grad_L_dz = torch.ones(5):view(-1,1)
    local grad_L_W = torch.zeros(nfeatures[1]+1,5)
    
    local c_s = 1
    
    for ep = 1,ep_max do
        
        timer = torch.Timer()

        shuffle:copy(torch.randperm(Xtrain:size(1)):type('torch.LongTensor'))
        
        local tot_Loss = 0
        
        for it = 1,math.floor(Xtrain:size(1)/m) do

            
            grad_W:zero()
            grad_b:zero()

            for i = (m*(it-1)+1),m*it do

                --current sentence:
                c_s = shuffle[i]

                --evaluate z:
                z:copy(W:index(1,Xtrain[c_s]:type('torch.LongTensor')):sum(1):add(b)):view(-1,1)

                --evaluate y_hat:
                yhat = torch.exp(z)/(z:exp():sum())

                --evaluate the loss:
                if it == math.floor(Xtrain:size(1)/m) then
                    tot_Loss = tot_Loss - z[Ytrain[c_s]][1] + math.log((z-(z:max())):exp():sum())+z:max()
                end
                --evaluate the gradients:
                -- First with respect to dz (which is equal to db):

                grad_L_dz:copy(yhat)
                grad_L_dz[Ytrain[c_s]]:csub(1)


                --Then with respect to dW:
                grad_L_W:zero()
                grad_L_W:indexAdd(1,Xtrain[c_s]:type('torch.LongTensor'),torch.expand(grad_L_dz,5,53):t())
                grad_L_W[1]:zero()

                --Update the gradients and total Loss
                grad_W:add(grad_L_W*(1/m))
                grad_b:add(grad_L_dz*1/m)
                

            end

            --Update the parameters:
            if (it%10) == 0 then
                
                W:mul(1-eta*lambda/(W:nElement()+b:nElement())):csub(grad_W:mul(eta))
                b:mul(1-eta*lambda/(W:nElement()+b:nElement())):csub(grad_b:mul(eta))
                
                if it == math.floor(Xtrain:size(1)/m) then
                    tot_Loss = Xtrain:size(1)*tot_Loss/m  + (0.5)*lambda*(torch.pow(W,2):sum()+torch.pow(b,2):sum())
                end
                
            else
                W:csub(grad_W:mul(eta))
                b:csub(grad_b:mul(eta))
                
                if it == math.floor(Xtrain:size(1)/m) then
                    tot_Loss = Xtrain:size(1)*tot_Loss/m  + (0.5)*lambda*(torch.pow(W,2):sum()+torch.pow(b,2):sum())
                end
                
            end
            
        end
        
        print('Time elapsed for epoch ' .. ep ..': ' .. timer:time().real .. ' seconds',"\n")
        print('Approximative Loss for the last batch for epoch ' .. ep ..': ' .. tot_Loss,"\n")
        print("\n")
        
    end
    
    return W,b,tot_Loss
    
end

In [4]:
function predict(Xvalid,W,b,Yvalid)
    local Yvalid = Yvalid or Nil
    local z = torch.zeros(5):view(-1,1)
    local Ypred = torch.IntTensor(Xvalid:size()[1])
    local max = torch.zeros(1)
    local accu = 0
    
    for i = 1,Xvalid:size()[1] do
        z:copy(W:index(1,Xvalid[i]:type('torch.LongTensor')):sum(1):add(b)):view(-1,1)
        max, Ypred[i] = z:max(1)
        
        if Yvalid then
            if Yvalid[i] == Ypred[i] then
                accu = accu + 1
            end
        end
    end
    
    if Yvalid then
        print("Accuracy: " .. accu/Xvalid:size()[1])
        return Ypred,accu/(Xvalid:size()[1])
    else
        return Ypred
    end
    
end

In [5]:
Hyperparam_ = torch.zeros(27,4)
m_  = torch.Tensor({10,20,50})
eta_ = torch.Tensor({0.1,1,10})
lambda_ = torch.Tensor({0.1,1,10})

for i = 0,2 do
    for z = 1,9 do
        Hyperparam_[9*i+z][1] = m_[i+1]
    end
end

for i = 1,3 do
        for z = 0,2 do
            Hyperparam_[z*9+i][2] = eta_[1]
        end
end

for i = 4,6 do
        for z = 0,2 do
            Hyperparam_[z*9+i][2] = eta_[2]
        end
end

for i = 7,9 do
        for z = 0,2 do
            Hyperparam_[z*9+i][2] = eta_[3]
        end
end

a = torch.Tensor({0.1,1,10})
for i =1,8 do
    a = torch.cat(a,torch.Tensor({0.1,1,10}),1)
end

Hyperparam_:narrow(2,3,1):copy(a)

In [ ]:
for i = 1,27 do
    print('Model '..i,'\n')
    torch.manualSeed(123)
    W, b, L = logreg(og_train_input,og_train_output,nfeatures,1,Hyperparam_[i][1],Hyperparam_[i][2],Hyperparam_[i][3])
    Validpred, Hyperparam_[i][4] = predict(og_valid_input,W,b,og_valid_output)
    print('\n')
end

Model 1	
	


In [ ]:
max,argmax = Hyperparam_:max(1)

In [ ]:
argmax

In [ ]:
Hyperparam_[25]

In [ ]:
math.randomseed(1234)
W, b, L = logreg(og_train_input,og_train_output,nfeatures,1,50,10,.1)

In [ ]:
Validpred, acc_val = predict(og_valid_input,W,b,og_valid_output)
Trainpred, acc_train  = predict(og_train_input,W,b,og_train_output)

In [ ]:
math.randomseed(1234)
W, b, L = logreg(og_train_input,og_train_output,nfeatures,1,20,1,0)

### Crossval SVM

In [ ]:
function linearSVM(Xtrain,Ytrain,nfeatures,ep_max,m,eta,lambda)
    local shuffle = torch.LongTensor(Xtrain:size(1))
    
    local W = torch.cat(torch.zeros(1,5),torch.ones(nfeatures[1],5),1)
    local b = torch.ones(5):view(-1,1)

    local grad_W = torch.zeros(nfeatures[1]+1,5)
    local grad_b = torch.zeros(5):view(-1,1)

    local yhat = torch.ones(5):view(-1,1)
    local y_temp = torch.zeros(5):view(-1,1)
    local grad_L_dy = torch.ones(5):view(-1,1)
    local grad_L_W = torch.zeros(nfeatures[1]+1,5)
    
    local c_s = 1
    
    for ep = 1,ep_max do
        
        timer = torch.Timer()

        shuffle:copy(torch.randperm(Xtrain:size(1)):type('torch.LongTensor'))
        
        local tot_Loss = 0
        
        for it = 1,math.floor(Xtrain:size(1)/m) do

            tot_Loss = 0
            grad_W:zero()
            grad_b:zero()

            for i = (m*(it-1)+1),m*it do

                --current sentence:
                c_s = shuffle[i]

                --evaluate y_hat:
                y_hat = W:index(1,Xtrain[c_s]:type('torch.LongTensor')):sum(1):add(b):view(-1,1)

                --evaluate the loss:
                
                y_temp:copy(y_hat)
                y_temp[Ytrain[c_s]]:fill(-9999)
                max_c,argmax_c = y_temp:max(1)
                
                if it == math.floor(Xtrain:size(1)/m) then
                    tot_Loss = tot_Loss + math.max(0,1-y_hat[Ytrain[c_s]][1]+max_c[1][1])
                end
                
                -- First with respect to dy (which is equal to db):

                grad_L_dy:zero()
                if (y_hat[Ytrain[c_s]][1]-max_c[1][1])<1 then
                    grad_L_dy[Ytrain[c_s]] = -1
                    grad_L_dy[argmax_c[1][1]] = 1
                end

                --evaluate the gradients:
                --Then with respect to dW:
                grad_L_W:zero()
                grad_L_W:indexAdd(1,Xtrain[c_s]:type('torch.LongTensor'),torch.expand(grad_L_dy,5,53):t())
                grad_L_W[1]:zero()

                --Update the gradients and total Loss
                grad_W:add(grad_L_W*(1/m))
                grad_b:add(grad_L_dy*(1/m))

            end

            --Update the parameters:
            
            if (it%10) == 0 then 
                W:mul(1-eta*lambda/(W:nElement()+b:nElement())):csub(grad_W:mul(eta))
                b:mul(1-eta*lambda/(W:nElement()+b:nElement())):csub(grad_b:mul(eta))
                
                if it == math.floor(Xtrain:size(1)/m) then
                    tot_Loss = Xtrain:size(1)*tot_Loss/m  + (0.5)*lambda*(torch.pow(W,2):sum()+torch.pow(b,2):sum())
                end
                
            else
                W:csub(grad_W:mul(eta))
                b:csub(grad_b:mul(eta))
                if it == math.floor(Xtrain:size(1)/m) then
                    tot_Loss = Xtrain:size(1)*tot_Loss/m + (0.5)*lambda*(torch.pow(W,2):sum()+torch.pow(b,2):sum())
                end
            end
            
        end
        
        print('Time elapsed for epoch ' .. ep ..': ' .. timer:time().real .. ' seconds',"\n")
        print('Approximative Loss for epoch ' .. ep ..': ' .. tot_Loss,"\n")
        print("\n")
        
    end
    
    return W,b,tot_Loss
end

In [ ]:
Hyperparam = torch.zeros(27,4)
m_  = torch.Tensor({10,20,50})
eta_ = torch.Tensor({0.1,1,10})
lambda_ = torch.Tensor({0.1,1,10})

for i = 0,2 do
    for z = 1,9 do
        Hyperparam[9*i+z][1] = m_[i+1]
    end
end

for i = 1,3 do
        for z = 0,2 do
            Hyperparam[z*9+i][2] = eta_[1]
        end
end

for i = 4,6 do
        for z = 0,2 do
            Hyperparam[z*9+i][2] = eta_[2]
        end
end

for i = 7,9 do
        for z = 0,2 do
            Hyperparam[z*9+i][2] = eta_[3]
        end
end

a = torch.Tensor({0.1,1,10})
for i =1,8 do
    a = torch.cat(a,torch.Tensor({0.1,1,10}),1)
end

Hyperparam:narrow(2,3,1):copy(a)


In [ ]:
for i = 1,27 do
    print('Model '..i,'\n')
    math.randomseed(1234)
    Ws, bs, Ls = linearSVM(og_train_input,og_train_output,nfeatures,1,Hyperparam[i][1],Hyperparam[i][2],Hyperparam[i][3])
    Validpreds, Hyperparam[i][4] = predict(og_valid_input,Ws,bs,og_valid_output)
    print('\n')
end

In [ ]:
max,argmax = Hyperparam:max(1)

In [ ]:
argmax[1][4]

In [ ]:
Hyperparam[22]

In [ ]:
math.randomseed(1234)
Ws, bs, Ls = linearSVM(og_train_input,og_train_output,nfeatures,3,50,1,0.1)
Validpreds, accus = predict(og_valid_input,Ws,bs,og_valid_output)

In [ ]:
math.randomseed(1234)
Ws, bs, Ls = linearSVM(og_train_input,og_train_output,nfeatures,5,50,1,10)
Validpreds, accus = predict(og_valid_input,Ws,bs,og_valid_output)

### LOGREG on test with best combi from CV

In [ ]:
math.randomseed(1234)
W, b, L = logreg(og_train_input,og_train_output,nfeatures,1,Hyperparam_[25][1],Hyperparam_[25][2],Hyperparam_[25][3])


In [ ]:
Testpred = predict(og_test_input,W,b)

In [ ]:
myFile = hdf5.open('Testpred_1.h5', 'w')
myFile:write('Testpred', Testpred)
myFile:close()

In [3]:
function logreg(Xtrain, Ytrain, nfeatures, nclasses, ep_max, batch_size, eta, lambda)
    
    -- Initialization
    local nfeatures = nfeatures[1]
    local nclasses = nclasses[1]
    local shuffle = torch.LongTensor(Xtrain:size(1))
    local W = torch.cat(torch.zeros(1,nclasses),torch.ones(nfeatures, nclasses),1)
    local b = torch.ones(nclasses, 1)
    local grad_W = torch.zeros(nfeatures+1,nclasses)
    local grad_b = torch.zeros(nclasses, 1)
    local z = torch.ones(nclasses, 1)
    local yhat = torch.ones(nclasses, 1)
    local grad_L_dz = torch.ones(nclasses, 1)
    local grad_L_W = torch.zeros(nfeatures+1,nclasses)
    

    
    local c_s = 1
    local it_max = math.floor(Xtrain:size(1)/batch_size)
    
    for ep = 1,ep_max do
        timer = torch.Timer()
        shuffle:copy(torch.randperm(Xtrain:size(1)):type('torch.LongTensor'))
        tot_Loss = 0
        
        for it = 1,it_max do
            -- Initializing gradient for the current iteration
            grad_W:zero()
            grad_b:zero()

            for i = (batch_size*(it-1)+1),batch_size*it do
                --current sentence:
                c_s = shuffle[i]
                --evaluate z:
                z:copy(W:index(1,Xtrain[c_s]:type('torch.LongTensor')):sum(1):add(b)):view(-1,1)
                --evaluate y_hat:
                yhat:copy(torch.exp(z))
                yhat:div(yhat:sum())

                 --evaluate the loss (only on the last iteration for the logger)
                if it == it_max then
                    tot_Loss = tot_Loss - z[Ytrain[c_s]][1] + math.log((z-(z:max())):exp():sum())+z:max()
                end
                --evaluate the gradients:
                -- First with respect to dz (which is equal to db):
                grad_L_dz:copy(yhat)
                grad_L_dz[Ytrain[c_s]]:csub(1)
                --Then with respect to dW:
                grad_L_W:zero()
                grad_L_W:indexAdd(1,Xtrain[c_s]:type('torch.LongTensor'),torch.expand(grad_L_dz,nclasses,53):t())
                grad_L_W[1]:zero()
                --Update the gradients
                grad_W:add(grad_L_W*(1/batch_size))
                grad_b:add(grad_L_dz*(1/batch_size))
            end

            -- Apply the regularization every 10 iteratin to gain speed (possible because of the dataset sparsity)
            if (it%10) == 0 then
                W:mul(1-eta*lambda/(W:nElement()+b:nElement())):csub(grad_W:mul(eta))
                b:mul(1-eta*lambda/(W:nElement()+b:nElement())):csub(grad_b:mul(eta))
            else
                W:csub(grad_W:mul(eta))
                b:csub(grad_b:mul(eta))
            end
            -- Updating the loss with the regularization
            if it == it_max then
                tot_Loss = Xtrain:size(1)*tot_Loss/batch_size + (0.5)*lambda*(torch.pow(W,2):sum()+torch.pow(b,2):sum())
            end
        end      
        print('Time elapsed for epoch ' .. ep ..': ' .. timer:time().real .. ' seconds')
        print('Approximative Loss for the last batch for epoch ' .. ep ..': ' .. tot_Loss)
    end
    return W, b, tot_Loss
    
end

In [4]:
math.randomseed(1234)
W,b,L = logreg(og_train_input, og_train_output, nfeatures, nclasses, 1, 50, 10, .1)

Time elapsed for epoch 1: 97.107171058655 seconds	
Approximative Loss for the last batch for epoch 1: 269419.08690127	


In [6]:
Validpred, accu = predict(og_valid_input,W,b,og_valid_output)

Accuracy: 0.38328792007266	
